# Claude Code SDK Autonomous Multi-Agent System

This notebook demonstrates how to use the autonomous multi-agent system to execute development tasks using Claude Code SDK.

## System Overview

- **Project Manager**: Analyzes tasks and delegates to appropriate agents
- **Frontend Coder**: Handles React, Vue.js, and frontend development
- **Backend Coder**: Handles Python, Node.js, and backend development  
- **DevOps Coder**: Handles Docker, CI/CD, and infrastructure

## Setup and Dependencies

**Important**: Before running this notebook, make sure you have Claude Code SDK installed and properly configured.

In [1]:
# Install required dependencies and apply asyncio support for Jupyter
!pip install claude-code-sdk --quiet
!pip install nest-asyncio --quiet

# Apply nest_asyncio for Jupyter compatibility
import nest_asyncio
nest_asyncio.apply()
print("✅ Dependencies installed and asyncio configured for Jupyter")

No global/local python version has been set yet. Please set the global/local version by typing:
pyenv global 3.7.4
pyenv local 3.7.4
No global/local python version has been set yet. Please set the global/local version by typing:
pyenv global 3.7.4
pyenv local 3.7.4
✅ Dependencies installed and asyncio configured for Jupyter


## Critical Requirement: Install Claude Code CLI

**IMPORTANT**: This system requires both the Python SDK and the Claude Code CLI. Please install the Claude Code CLI first:

```bash
npm install -g @anthropic-ai/claude-code
```

After installation, you may need to restart this notebook kernel to ensure the CLI is available in the PATH.

In [2]:
# Verify Claude Code CLI installation
import subprocess
import os

print("🔍 Checking Claude Code CLI installation...")

try:
    # Check if claude command is available
    result = subprocess.run(['claude', '--version'], capture_output=True, text=True, timeout=10)
    if result.returncode == 0:
        print(f"✅ Claude Code CLI found: {result.stdout.strip()}")
    else:
        print(f"❌ Claude Code CLI command failed: {result.stderr}")
except FileNotFoundError:
    print("❌ Claude Code CLI not found in PATH")
    print("   Please install with: npm install -g @anthropic-ai/claude-code")
except Exception as e:
    print(f"❌ Error checking CLI: {e}")

# Check PATH
print(f"\n📍 Current PATH directories:")
path_dirs = os.environ.get('PATH', '').split(os.pathsep)
for i, path_dir in enumerate(path_dirs[:5]):  # Show first 5
    print(f"   {i+1}. {path_dir}")
if len(path_dirs) > 5:
    print(f"   ... and {len(path_dirs) - 5} more directories")

# Check if Node.js and npm are available (required for Claude CLI)
print(f"\n🟢 Checking Node.js environment:")
try:
    node_result = subprocess.run(['node', '--version'], capture_output=True, text=True)
    print(f"   Node.js: {node_result.stdout.strip() if node_result.returncode == 0 else 'Not found'}")
    
    npm_result = subprocess.run(['npm', '--version'], capture_output=True, text=True)
    print(f"   npm: {npm_result.stdout.strip() if npm_result.returncode == 0 else 'Not found'}")
except:
    print("   Node.js/npm not found")

print(f"\n💡 If Claude CLI is not installed, please run:")
print(f"   npm install -g @anthropic-ai/claude-code")
print(f"   Then restart this notebook kernel.")

🔍 Checking Claude Code CLI installation...
❌ Claude Code CLI not found in PATH
   Please install with: npm install -g @anthropic-ai/claude-code

📍 Current PATH directories:
   1. c:\Users\johnny\AppData\Local\Programs\Python\Python311
   2. c:\Users\johnny\AppData\Roaming\Python\Python311\Scripts
   3. C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.6\bin
   4. C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.6\libnvvp
   5. C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.3\bin
   ... and 95 more directories

🟢 Checking Node.js environment:
   Node.js: v22.17.0
   Node.js/npm not found

💡 If Claude CLI is not installed, please run:
   npm install -g @anthropic-ai/claude-code
   Then restart this notebook kernel.


In [3]:
# Import required libraries and setup paths
import sys
import os
import time
import asyncio
import datetime
import json

# Setup working directory and paths
current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")

# Ensure we're in the right directory and setup Python path correctly
if not current_dir.endswith('/agent'):
    # If not in agent directory, navigate to it
    if 'src' in current_dir:
        agent_dir = current_dir
    else:
        agent_dir = os.path.join(current_dir, 'src', 'agent')
    os.chdir(agent_dir)

# Add both current directory and parent to Python path
agent_dir = os.getcwd()
src_dir = os.path.dirname(agent_dir)
project_root = os.path.dirname(src_dir)

sys.path.insert(0, agent_dir)
sys.path.insert(0, src_dir)
sys.path.insert(0, project_root)

print(f"Working directory: {os.getcwd()}")
print(f"Python path setup: {sys.path[:3]}")

# Clear any cached imports to force reload
modules_to_clear = [
    'utils', 'claude_sdk_interface', 'system_coordinator', 
    'specialized_agents', 'models', 'base_agent'
]
for module in modules_to_clear:
    if module in sys.modules:
        del sys.modules[module]
        print(f"Cleared cached module: {module}")

# Fix relative import issues by using absolute imports
try:
    # Import all modules with absolute imports to avoid relative import errors
    print("Importing modules...")
    import models
    print("✅ models imported")
    import claude_sdk_interface
    print("✅ claude_sdk_interface imported")
    import base_agent
    print("✅ base_agent imported")
    import specialized_agents
    print("✅ specialized_agents imported")
    import system_coordinator
    print("✅ system_coordinator imported")
    import utils
    print("✅ utils imported")
    
    print(f"📦 Claude SDK Available: {claude_sdk_interface.is_sdk_available()}")
    
    # Import main functions
    from utils import init_autonomous_system, get_system_info
    from claude_sdk_interface import is_sdk_available
    
    print("✅ Successfully imported multi-agent system functions")
    
except ImportError as e:
    print(f"❌ Import error: {e}")
    import traceback
    traceback.print_exc()
    print("Please ensure all files are present and you're running from the correct directory")
    raise

Current working directory: d:\My Documents\auto-coding-multi-agent\src\agent
Working directory: d:\My Documents\auto-coding-multi-agent\src\agent
Python path setup: ['d:\\My Documents\\auto-coding-multi-agent', 'd:\\My Documents\\auto-coding-multi-agent\\src', 'd:\\My Documents\\auto-coding-multi-agent\\src\\agent']
Importing modules...
✅ models imported
✅ claude_sdk_interface imported
✅ base_agent imported
✅ specialized_agents imported
✅ system_coordinator imported
✅ utils imported
📦 Claude SDK Available: True
✅ Successfully imported multi-agent system functions


## System Initialization

In [4]:
# Initialize and start the multi-agent system
try:
    # Initialize system
    print("🔧 Initializing multi-agent system...")
    system = await init_autonomous_system()
    
    print("🚀 Starting agents...")
    await system.start_system()
    
    print("✅ Multi-agent system started successfully!")
    
    # Display system information
    system_info = get_system_info()
    print(f"\n📋 System Info:")
    print(f"  - SDK Available: {system_info['sdk_available']}")
    print(f"  - Supported Agents: {', '.join(system_info['supported_agents'])}")
    print(f"  - Features: {', '.join(system_info['features'])}")
    
    # Ensure output directory exists
    output_dir = "../../output"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"📁 Created output directory: {output_dir}")
    
    # Show initial dashboard
    system.display_system_dashboard()
    
except Exception as e:
    print(f"❌ System initialization failed: {e}")
    import traceback
    traceback.print_exc()
    
    # Provide troubleshooting information
    print(f"\n🔍 Troubleshooting Info:")
    print(f"  - Working Directory: {os.getcwd()}")
    print(f"  - Python Path: {sys.path[:2]}")
    print(f"  - Available Files: {os.listdir('.')}")
    
    raise

🔧 Initializing multi-agent system...
✅ Claude Code SDK autonomous multi-agent system initialized
🔧 Available agents: ['project_manager', 'frontend_coder', 'backend_coder', 'devops_coder']
🚀 Starting agents...
✅ Multi-agent system started successfully!

📋 System Info:
  - SDK Available: True
  - Supported Agents: project_manager, frontend_coder, backend_coder, devops_coder
  - Features: Autonomous task execution, Multi-agent coordination, Real-time progress monitoring, Structured task creation

🎆 Autonomous Multi-Agent System Dashboard
🟢 System Status: Running
🤖 Active Agents: 4
✅ Completed Tasks: 0
⏳ Active Tasks: 0
📝 Pending Projects: 0
🏆 Completed Projects: 0

👥 Agent Details:
------------------------------------------------------------
🟢 project_manager (project_manager)
    ✅ Completed: 0, ⏳ Active: 0, 📨 Pending: 0
🟢 frontend_coder (frontend_coder)
    ✅ Completed: 0, ⏳ Active: 0, 📨 Pending: 0
🟢 backend_coder (backend_coder)
    ✅ Completed: 0, ⏳ Active: 0, 📨 Pending: 0
🟢 devops_co

ERROR:root:Claude Code SDK query failed: Claude Code SDK query failed: Claude Code not found. Install with:
  npm install -g @anthropic-ai/claude-code

If already installed locally, try:
  export PATH="$HOME/node_modules/.bin:$PATH"

Or specify the path when creating transport:
  SubprocessCLITransport(..., cli_path='/path/to/claude')
ERROR:root:Claude Code SDK execution failed: Claude Code SDK query failed: Claude Code not found. Install with:
  npm install -g @anthropic-ai/claude-code

If already installed locally, try:
  export PATH="$HOME/node_modules/.bin:$PATH"

Or specify the path when creating transport:
  SubprocessCLITransport(..., cli_path='/path/to/claude')
ERROR:root:Claude Code SDK query failed: Claude Code SDK query failed: Claude Code not found. Install with:
  npm install -g @anthropic-ai/claude-code

If already installed locally, try:
  export PATH="$HOME/node_modules/.bin:$PATH"

Or specify the path when creating transport:
  SubprocessCLITransport(..., cli_path='/pa

## Simple Task Examples

Let's demonstrate the multi-agent system with three practical examples. Each task will be analyzed by the Project Manager and delegated to the appropriate specialized agents.

**Note**: If tasks fail, this usually indicates Claude Code SDK connectivity issues or import problems. Check the system initialization output above for any error messages.

In [5]:
# Quick system test to verify components are working
print("🔧 Testing system components...")

# Test 1: Check agents are initialized
print(f"✅ System running: {system.is_running}")
print(f"✅ Agents available: {list(system.all_agents.keys())}")

# Test 2: Check Project Manager can create fallback tasks
try:
    print("\n🧪 Testing project manager task analysis...")
    test_task = "Create a simple hello world Python script"
    fallback_tasks = system.project_manager._create_fallback_task(test_task)
    print(f"✅ Fallback task creation works: {len(fallback_tasks)} task(s) created")
    if fallback_tasks:
        print(f"   - Task assigned to: {fallback_tasks[0].assigned_to}")
        print(f"   - Task description: {fallback_tasks[0].description[:50]}...")
except Exception as e:
    print(f"❌ Project manager test failed: {e}")

# Test 3: Check output directory exists
import os
output_dir = "../../output"
if os.path.exists(output_dir):
    print(f"✅ Output directory exists: {output_dir}")
    files = os.listdir(output_dir)
    print(f"   - Current files: {len(files)} files")
else:
    print(f"⚠️  Output directory does not exist, will be created: {output_dir}")

# Test 4: Check Claude Code SDK status
from claude_sdk_interface import is_sdk_available
sdk_status = is_sdk_available()
print(f"📦 Claude Code SDK available: {sdk_status}")

print("\n🎯 System ready for task execution!")
print("=" * 50)

🔧 Testing system components...
✅ System running: True
✅ Agents available: ['project_manager', 'frontend_coder', 'backend_coder', 'devops_coder']

🧪 Testing project manager task analysis...
✅ Fallback task creation works: 1 task(s) created
   - Task assigned to: backend_coder
   - Task description: Create a simple hello world Python script...
✅ Output directory exists: ../../output
   - Current files: 0 files
📦 Claude Code SDK available: True

🎯 System ready for task execution!


## System Debugging and Testing

Before running the main examples, let's test the system components to ensure they're working properly.

In [6]:
# Direct test: Try to execute a simple Claude Code SDK query manually
print("🧪 Testing Claude Code SDK directly...")

try:
    from claude_sdk_interface import execute_claude_query, create_claude_options
    
    # Create simple test options
    test_options = create_claude_options(
        allowed_tools=["Write", "Read"],
        system_prompt="You are a helpful assistant.",
        max_turns=3
    )
    
    # Simple test prompt
    test_prompt = """
Please create a simple test file called '../../output/test_file.txt' with the content 'Hello from Claude Code SDK test'.

Use the Write tool to create this file.
"""
    
    print("Sending test query to Claude Code SDK...")
    response_parts = []
    
    async for msg in execute_claude_query(test_prompt, test_options):
        from claude_sdk_interface import AssistantMessage, TextBlock, ResultMessage
        if isinstance(msg, AssistantMessage):
            for block in msg.content:
                if isinstance(block, TextBlock):
                    response_parts.append(block.text)
                    print(f"📝 SDK Response: {block.text}")
        elif isinstance(msg, ResultMessage):
            print("✅ SDK query completed")
            break
    
    # Check if test file was created
    import os
    test_file_path = "../../output/test_file.txt"
    if os.path.exists(test_file_path):
        print("✅ Test file created successfully!")
        with open(test_file_path, 'r') as f:
            content = f.read()
            print(f"📄 File content: {content}")
    else:
        print("❌ Test file was not created")
        print("This indicates Claude Code SDK is not executing tools")
    
except Exception as e:
    print(f"❌ Direct SDK test failed: {e}")
    import traceback
    traceback.print_exc()

🧪 Testing Claude Code SDK directly...
Sending test query to Claude Code SDK...
❌ Direct SDK test failed: Claude Code SDK query failed: Claude Code not found. Install with:
  npm install -g @anthropic-ai/claude-code

If already installed locally, try:
  export PATH="$HOME/node_modules/.bin:$PATH"

Or specify the path when creating transport:
  SubprocessCLITransport(..., cli_path='/path/to/claude')


Traceback (most recent call last):
  File "d:\My Documents\auto-coding-multi-agent\src\agent\claude_sdk_interface.py", line 69, in execute_claude_query
    async for message in query(prompt=prompt, options=options):
  File "c:\Users\johnny\AppData\Local\Programs\Python\Python311\Lib\site-packages\claude_code_sdk\query.py", line 101, in query
    async for message in client.process_query(prompt=prompt, options=options):
  File "c:\Users\johnny\AppData\Local\Programs\Python\Python311\Lib\site-packages\claude_code_sdk\_internal\client.py", line 22, in process_query
    transport = SubprocessCLITransport(
                ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\johnny\AppData\Local\Programs\Python\Python311\Lib\site-packages\claude_code_sdk\_internal\transport\subprocess_cli.py", line 41, in __init__
    self._cli_path = str(cli_path) if cli_path else self._find_cli()
                                                    ^^^^^^^^^^^^^^^^
  File "c:\Users\johnny\AppData\Local\Programs\Python\

In [7]:
# Debug: Check the detailed results from the backend_coder
if hasattr(system.coding_agents['backend_coder'], 'recent_task_results'):
    results = system.coding_agents['backend_coder'].recent_task_results
    print("🔍 Backend Coder Recent Results:")
    for i, result in enumerate(results):
        print(f"\n📋 Result {i+1}:")
        print(f"  - Agent: {result.get('agent_id', 'unknown')}")
        print(f"  - Task: {result.get('task_description', '')[:100]}...")
        print(f"  - Files Created: {len(result.get('created_files', []))}")
        print(f"  - Files: {result.get('created_files', [])}")
        if 'error' in result:
            print(f"  - Error: {result['error']}")
        if 'warning' in result:
            print(f"  - Warning: {result['warning']}")
        if 'response' in result:
            print(f"  - Response Length: {len(result['response'])} characters")
            print(f"  - Response Preview: {result['response'][:200]}...")

# Check the latest project results
projects = system.get_project_results()
if projects:
    latest = projects[-1]
    print(f"\n🎯 Latest Project Details:")
    print(f"  - Status: {latest.get('status')}")
    print(f"  - Subtasks: {len(latest.get('subtasks', []))}")
    print(f"  - Results: {len(latest.get('results', []))}")
    print(f"  - Created Files: {latest.get('created_files', [])}")
    if 'error' in latest:
        print(f"  - Error: {latest['error']}")
    
    # Show subtask details
    for i, subtask in enumerate(latest.get('subtasks', [])):
        print(f"\n  📋 Subtask {i+1}:")
        print(f"    - ID: {subtask['id']}")
        print(f"    - Assigned to: {subtask['assigned_to']}")
        print(f"    - Description: {subtask['description'][:100]}...")

### Example 1: Create a Simple Python Calculator

In [8]:
# Task 1: Simple Python Calculator
calculator_task = """
Create a simple Python calculator application with the following features:
- Basic arithmetic operations (add, subtract, multiply, divide)
- Error handling for division by zero
- User-friendly command-line interface
- Save the calculator.py file to the output directory
- Include proper documentation and comments
"""

print("🧮 Task 1: Creating Python Calculator")
print("=" * 50)

start_time = datetime.datetime.now()
print(f"📅 Started at: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

try:
    # Submit task to the system
    await system.submit_project_task(calculator_task)
    print("📝 Task submitted to multi-agent system")

    # Wait for completion with progress updates
    print("⏳ Waiting for agents to complete the task...")
    
    # Show system status while waiting
    await asyncio.sleep(2)  # Give system time to process
    status = system.get_system_status()
    print(f"📊 Current Status: {status.total_active_tasks} active tasks, {status.pending_projects} pending projects")
    
    result = await system.wait_for_completion(timeout_minutes=10)

    end_time = datetime.datetime.now()
    duration = (end_time - start_time).total_seconds()

    print(f"\n✅ Task processing completed in {duration:.1f} seconds")
    print(f"📅 Finished at: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")

    # Show results
    projects = system.get_project_results()
    if projects:
        latest = projects[-1]
        print(f"\n📊 Results:")
        print(f"  - Status: {latest.get('status', 'unknown')}")
        print(f"  - Subtasks: {len(latest.get('subtasks', []))}")
        print(f"  - Agent Results: {len(latest.get('results', []))}")
        
        # Show created files if any
        if 'created_files' in latest and latest['created_files']:
            print(f"  - Created Files: {', '.join(latest['created_files'])}")
    
    # Show updated dashboard
    system.display_system_dashboard()
    
except Exception as e:
    print(f"❌ Task execution failed: {e}")
    import traceback
    traceback.print_exc()
    
    # Show system status for debugging
    try:
        system.display_system_dashboard()
    except:
        print("Could not display dashboard")

🧮 Task 1: Creating Python Calculator
📅 Started at: 2025-08-30 07:40:42
📝 Task submitted to multi-agent system
⏳ Waiting for agents to complete the task...
📊 Current Status: 0 active tasks, 1 pending projects

✅ Task processing completed in 6.0 seconds
📅 Finished at: 2025-08-30 07:40:48

📊 Results:
  - Status: failed
  - Subtasks: 1
  - Agent Results: 0

🎆 Autonomous Multi-Agent System Dashboard
🟢 System Status: Running
🤖 Active Agents: 4
✅ Completed Tasks: 1
⏳ Active Tasks: 0
📝 Pending Projects: 0
🏆 Completed Projects: 1

👥 Agent Details:
------------------------------------------------------------
🟢 project_manager (project_manager)
    ✅ Completed: 0, ⏳ Active: 0, 📨 Pending: 0
🟢 frontend_coder (frontend_coder)
    ✅ Completed: 0, ⏳ Active: 0, 📨 Pending: 0
🟢 backend_coder (backend_coder)
    ✅ Completed: 1, ⏳ Active: 0, 📨 Pending: 0
    📈 Recent Results: 0 files
🟢 devops_coder (devops_coder)
    ✅ Completed: 0, ⏳ Active: 0, 📨 Pending: 0

📅 Last Updated: 2025-08-30T07:40:48.889857



### Example 2: Create a Simple Web API

In [9]:
# Task 2: Simple REST API
api_task = """
Create a simple REST API using Python Flask with the following features:
- GET /api/hello endpoint that returns a JSON greeting
- POST /api/echo endpoint that echoes back the request data
- Basic error handling and status codes
- Save as app.py in the output directory
- Include requirements.txt with dependencies
- Add basic documentation
"""

print("🌐 Task 2: Creating REST API")
print("=" * 50)

start_time = datetime.datetime.now()
print(f"📅 Started at: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

try:
    # Submit task
    await system.submit_project_task(api_task)
    print("📝 Task submitted to multi-agent system")

    # Wait for completion
    print("⏳ Waiting for agents to complete the task...")
    
    # Progress check
    await asyncio.sleep(2)
    status = system.get_system_status()
    print(f"📊 Current Status: {status.total_active_tasks} active tasks, {status.pending_projects} pending projects")
    
    result = await system.wait_for_completion(timeout_minutes=10)

    end_time = datetime.datetime.now()
    duration = (end_time - start_time).total_seconds()

    print(f"\n✅ Task processing completed in {duration:.1f} seconds")

    # Show results
    projects = system.get_project_results()
    if projects:
        latest = projects[-1]
        print(f"\n📊 Latest Project Results:")
        print(f"  - Status: {latest.get('status', 'unknown')}")
        if 'created_files' in latest and latest['created_files']:
            print(f"  - Created Files: {', '.join(latest['created_files'])}")

    # Show updated dashboard
    system.display_system_dashboard()
    
except Exception as e:
    print(f"❌ Task execution failed: {e}")
    print("Continuing with next example...")

🌐 Task 2: Creating REST API
📅 Started at: 2025-08-30 07:40:48
📝 Task submitted to multi-agent system
⏳ Waiting for agents to complete the task...
📊 Current Status: 0 active tasks, 1 pending projects

✅ Task processing completed in 4.0 seconds

📊 Latest Project Results:
  - Status: failed

🎆 Autonomous Multi-Agent System Dashboard
🟢 System Status: Running
🤖 Active Agents: 4
✅ Completed Tasks: 2
⏳ Active Tasks: 0
📝 Pending Projects: 0
🏆 Completed Projects: 2

👥 Agent Details:
------------------------------------------------------------
🟢 project_manager (project_manager)
    ✅ Completed: 0, ⏳ Active: 0, 📨 Pending: 0
🟢 frontend_coder (frontend_coder)
    ✅ Completed: 1, ⏳ Active: 0, 📨 Pending: 0
    📈 Recent Results: 0 files
🟢 backend_coder (backend_coder)
    ✅ Completed: 1, ⏳ Active: 0, 📨 Pending: 0
    📈 Recent Results: 0 files
🟢 devops_coder (devops_coder)
    ✅ Completed: 0, ⏳ Active: 0, 📨 Pending: 0

📅 Last Updated: 2025-08-30T07:40:52.916416



### Example 3: Create a Simple React Component

In [10]:
# Task 3: React Component
react_task = """
Create a simple React counter component with the following features:
- Counter component that displays a number
- Increment and decrement buttons
- Reset button to set counter to zero
- Modern React hooks (useState)
- Basic CSS styling
- Save as Counter.js and Counter.css in output directory
- Include package.json with React dependencies
"""

print("⚛️ Task 3: Creating React Counter Component")
print("=" * 50)

start_time = datetime.datetime.now()
print(f"📅 Started at: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

try:
    # Submit task
    await system.submit_project_task(react_task)
    print("📝 Task submitted to multi-agent system")

    # Wait for completion
    print("⏳ Waiting for agents to complete the task...")
    
    # Progress check
    await asyncio.sleep(2)
    status = system.get_system_status()
    print(f"📊 Current Status: {status.total_active_tasks} active tasks, {status.pending_projects} pending projects")
    
    result = await system.wait_for_completion(timeout_minutes=10)

    end_time = datetime.datetime.now()
    duration = (end_time - start_time).total_seconds()

    print(f"\n✅ Task processing completed in {duration:.1f} seconds")

    # Show results
    projects = system.get_project_results()
    if projects:
        latest = projects[-1]
        print(f"\n📊 Latest Project Results:")
        print(f"  - Status: {latest.get('status', 'unknown')}")
        if 'created_files' in latest and latest['created_files']:
            print(f"  - Created Files: {', '.join(latest['created_files'])}")

    # Show final dashboard
    system.display_system_dashboard()
    
except Exception as e:
    print(f"❌ Task execution failed: {e}")
    print("Please check system logs for details")

⚛️ Task 3: Creating React Counter Component
📅 Started at: 2025-08-30 07:40:52
📝 Task submitted to multi-agent system
⏳ Waiting for agents to complete the task...
📊 Current Status: 0 active tasks, 1 pending projects

✅ Task processing completed in 6.0 seconds

📊 Latest Project Results:
  - Status: failed

🎆 Autonomous Multi-Agent System Dashboard
🟢 System Status: Running
🤖 Active Agents: 4
✅ Completed Tasks: 3
⏳ Active Tasks: 0
📝 Pending Projects: 0
🏆 Completed Projects: 3

👥 Agent Details:
------------------------------------------------------------
🟢 project_manager (project_manager)
    ✅ Completed: 0, ⏳ Active: 0, 📨 Pending: 0
🟢 frontend_coder (frontend_coder)
    ✅ Completed: 2, ⏳ Active: 0, 📨 Pending: 0
    📈 Recent Results: 0 files, 0 files
🟢 backend_coder (backend_coder)
    ✅ Completed: 1, ⏳ Active: 0, 📨 Pending: 0
    📈 Recent Results: 0 files
🟢 devops_coder (devops_coder)
    ✅ Completed: 0, ⏳ Active: 0, 📨 Pending: 0

📅 Last Updated: 2025-08-30T07:40:58.907811



## Check Generated Files

In [11]:
# Check what files were created in the output directory
import os

output_dir = "../../output"
print("📁 Files created by the multi-agent system:")
print("=" * 50)

if os.path.exists(output_dir):
    # Walk through output directory
    file_count = 0
    for root, dirs, files in os.walk(output_dir):
        level = root.replace(output_dir, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}📁 {os.path.basename(root)}/")
        
        subindent = ' ' * 2 * (level + 1)
        for file in files:
            file_path = os.path.join(root, file)
            size = os.path.getsize(file_path)
            print(f"{subindent}📄 {file} ({size} bytes)")
            file_count += 1
    
    print(f"\n📊 Total files created: {file_count}")
else:
    print("❌ Output directory not found")
    print("This might indicate that agents haven't created files yet")

# Show project results summary
projects = system.get_project_results()
print(f"\n📈 Project Summary:")
print(f"  - Total projects completed: {len(projects)}")

for i, project in enumerate(projects, 1):
    print(f"  - Project {i}: {project.get('status', 'unknown')} status")
    if 'created_files' in project:
        print(f"    Files: {len(project['created_files'])}")

📁 Files created by the multi-agent system:
📁 output/

📊 Total files created: 0

📈 Project Summary:
  - Total projects completed: 3
  - Project 1: failed status
    Files: 0
  - Project 2: failed status
    Files: 0
  - Project 3: failed status
    Files: 0


## Custom Task Example

Try your own custom task! Modify the task description below and run the cell.

In [12]:
# Custom task - modify this as needed
custom_task = """
Create a simple Python script that:
- Reads a CSV file 
- Calculates basic statistics (mean, median, sum)
- Outputs the results to a text file
- Includes error handling for file operations
- Save as data_analyzer.py in the output directory
"""

print("🔧 Custom Task: Data Analyzer")
print("=" * 50)
print(f"Task: {custom_task.strip()}")

# Interactive execution (you can modify this to run automatically)
print("\n💡 Tip: You can modify the custom_task variable above and re-run this cell")
print("📝 Or run the task automatically by uncommenting the code below:")

# Uncomment the following code to run the task automatically:
# start_time = datetime.datetime.now()
# print(f"\n📅 Started at: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

# try:
#     await system.submit_project_task(custom_task)
#     print("📝 Custom task submitted")
    
#     result = await system.wait_for_completion(timeout_minutes=10)
    
#     end_time = datetime.datetime.now()
#     duration = (end_time - start_time).total_seconds()
    
#     print(f"\n✅ Custom task completed in {duration:.1f} seconds")
#     system.display_system_dashboard()
# except Exception as e:
#     print(f"❌ Custom task failed: {e}")

print("⏭️ Custom task ready for execution (uncomment code above to run)")

🔧 Custom Task: Data Analyzer
Task: Create a simple Python script that:
- Reads a CSV file 
- Calculates basic statistics (mean, median, sum)
- Outputs the results to a text file
- Includes error handling for file operations
- Save as data_analyzer.py in the output directory

💡 Tip: You can modify the custom_task variable above and re-run this cell
📝 Or run the task automatically by uncommenting the code below:
⏭️ Custom task ready for execution (uncomment code above to run)


## System Shutdown

In [13]:
# Stop the multi-agent system
try:
    print("🛑 Stopping multi-agent system...")
    await system.stop_system()
    print("✅ Multi-agent system stopped successfully")
    
    # Final summary
    projects = system.get_project_results()
    successful_projects = [p for p in projects if p.get('status') == 'completed']
    failed_projects = [p for p in projects if p.get('status') == 'failed']
    
    print(f"\n📊 Session Summary:")
    print(f"  - Total projects: {len(projects)}")
    print(f"  - Successful: {len(successful_projects)}")
    print(f"  - Failed: {len(failed_projects)}")
    print(f"  - Total files created: {len(system.get_created_files()) if hasattr(system, 'get_created_files') else len([f for p in projects for f in p.get('created_files', [])])}")
    print(f"  - Session ended: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Show any created files
    all_files = []
    for project in projects:
        if 'created_files' in project:
            all_files.extend(project['created_files'])
    
    if all_files:
        print(f"\n📁 Files Created This Session:")
        for file in all_files:
            print(f"    - {file}")
    else:
        print(f"\n⚠️  No files were created this session")
        print(f"   This might indicate agents need debugging or Claude Code SDK issues")
    
except Exception as e:
    print(f"❌ Error stopping system: {e}")
    
# Cleanup message
print(f"\n💡 Next Steps:")
print(f"   - Check the output/ directory for generated files")
print(f"   - Review any error messages above")
print(f"   - Restart the notebook if you want to run more tasks")

🛑 Stopping multi-agent system...
✅ Multi-agent system stopped successfully

📊 Session Summary:
  - Total projects: 3
  - Successful: 0
  - Failed: 3
  - Total files created: 0
  - Session ended: 2025-08-30 07:40:58

⚠️  No files were created this session
   This might indicate agents need debugging or Claude Code SDK issues

💡 Next Steps:
   - Check the output/ directory for generated files
   - Review any error messages above
   - Restart the notebook if you want to run more tasks


## Next Steps and Troubleshooting

### 🎉 Congratulations!
You've successfully used the Claude Code SDK Multi-Agent System. 

### 🔍 Troubleshooting

If you encountered issues:

1. **Import Errors**: Restart the kernel and ensure all cells are run in order
2. **Task Failures**: Check that Claude Code SDK is properly installed and accessible
3. **No Files Created**: Agents may need Claude Code SDK tools - verify SDK installation
4. **System Initialization Failed**: Check working directory and Python path setup

### 💡 Try More Complex Tasks

Modify the task descriptions in the examples above or create your own:

**Full-Stack Web Application**:
```python
complex_task = '''
Create a complete todo web application with:
- HTML/CSS/JavaScript frontend with modern design
- Python Flask backend with REST API
- SQLite database for data persistence
- CRUD operations (Create, Read, Update, Delete)
- Responsive design for mobile devices
- Include all necessary files and documentation
'''
```

**Data Analysis Pipeline**:
```python
analysis_task = '''
Create a data analysis pipeline that:
- Reads CSV data from multiple sources
- Cleans and preprocesses the data
- Performs statistical analysis
- Generates visualizations with matplotlib
- Exports results to Excel and PDF reports
- Includes comprehensive documentation
'''
```

**DevOps Infrastructure**:
```python
devops_task = '''
Create a complete DevOps setup including:
- Dockerfile for application containerization
- Docker Compose for multi-service orchestration
- GitHub Actions CI/CD pipeline
- Kubernetes deployment manifests
- Monitoring and logging configuration
- Documentation for deployment and maintenance
'''
```

### 🔧 System Features Recap

- **Project Manager**: Analyzes tasks and coordinates agent work
- **Frontend Coder**: React, Vue.js, HTML/CSS/JS development
- **Backend Coder**: Python, Node.js, API development
- **DevOps Coder**: Docker, CI/CD, infrastructure automation

Each agent uses Claude Code SDK tools to create real files with proper extensions in the output directory.